In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import datasets
from sklearn.metrics import confusion_matrix, accuracy_score
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import MinMaxScaler


In [36]:
import xgboost as xgb
import lightgbm as lgbm
import catboost as cbt
from sklearn_rvm import EMRVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Digits Dataset

In [38]:
# import some data to play with
digits = datasets.load_digits()


X = digits.data
y = digits.target

In [39]:
print("X shape")
print(X.shape)
print("Y shape")
print(y.shape)


X shape
(1797, 64)
Y shape
(1797,)


In [40]:
# x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=4096)
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=4096, train_size=0.8)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, random_state=4096, train_size=0.8)

In [41]:
scaler = MinMaxScaler()

scaler = scaler.fit(x_train)


In [42]:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_valid = scaler.transform(x_valid)

In [9]:
# data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
# target = raw_df.values[1::2, 2]
# x_train, x_test, y_train, y_test = train_test_split(data, target, random_state=4096)

# SVM - Linear

In [10]:

svm_classifier = SVC(kernel='linear')
svm_classifier = svm_classifier.fit(x_train, y_train)

In [11]:
y_pred = svm_classifier.predict(x_test)



print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))


Confusion Matrix
 [[40  0  0  0  1  0  0  0  0  0]
 [ 0 42  0  0  0  0  0  0  0  0]
 [ 0  0 38  0  0  0  0  0  0  0]
 [ 0  0  0 30  0  2  0  0  0  0]
 [ 0  0  0  0 36  0  0  0  0  0]
 [ 0  0  0  0  0 28  0  0  0  0]
 [ 0  0  0  0  1  0 25  0  0  0]
 [ 0  0  0  0  0  0  0 42  0  0]
 [ 0  1  0  0  0  0  0  0 35  0]
 [ 0  0  0  1  0  0  0  0  1 37]]
Accuracy  0.9805555555555555


# SVM - Kernel

In [12]:
param_grid = [
    {'kernel': ['linear'], 'C': [10., 30., 100., 300., 1000., 3000., 10000., 30000.0]},
    {'kernel': ['rbf'], 'C': [1.0, 3.0, 10., 30., 100., 300., 1000.0],
                        'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
]

In [13]:

svm_classifier = SVC(kernel='rbf')
# svm_classifier = svm_classifier.fit(x_train, y_train)

grid_search = GridSearchCV(svm_classifier, param_grid, cv=5, scoring="neg_mean_squared_error", verbose=2)
best_svm_classifier = grid_search.fit(x_train, y_train)



Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END ..............................C=10.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=10.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=10.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=10.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=10.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.0s
[CV] END .............................C=100.0, kernel=linear; total time=   0.0s
[CV] END .............................C=100.0, 

In [14]:
grid_search.best_params_


{'C': 3.0, 'gamma': 0.3, 'kernel': 'rbf'}

In [15]:
# y_pred = svm_classifier.predict(x_test)
y_pred = best_svm_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[41  0  0  0  0  0  0  0  0  0]
 [ 0 42  0  0  0  0  0  0  0  0]
 [ 0  0 38  0  0  0  0  0  0  0]
 [ 0  0  0 30  0  1  0  0  1  0]
 [ 0  0  0  0 36  0  0  0  0  0]
 [ 0  0  0  0  0 28  0  0  0  0]
 [ 0  0  0  0  0  0 26  0  0  0]
 [ 0  0  0  0  0  0  0 42  0  0]
 [ 0  0  0  0  0  0  0  0 36  0]
 [ 0  0  0  0  0  0  0  0  1 38]]
Accuracy  0.9916666666666667


# TabNet

In [17]:
tabnet_classifier = TabNetClassifier()
# clf = TabNetMultiTaskClassifier()
tabnet_classifier.fit(
  X_train=x_train, y_train=y_train,
  eval_set = [(x_valid, y_valid)],
  patience=1000,
  max_epochs=60,
  batch_size=16,
)

c:\venv_python_3.10\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 2.28333 | val_0_accuracy: 0.23611 |  0:00:02s
epoch 1  | loss: 1.89214 | val_0_accuracy: 0.37847 |  0:00:05s
epoch 2  | loss: 1.58083 | val_0_accuracy: 0.49653 |  0:00:08s
epoch 3  | loss: 1.3569  | val_0_accuracy: 0.66667 |  0:00:11s
epoch 4  | loss: 1.22514 | val_0_accuracy: 0.66667 |  0:00:14s
epoch 5  | loss: 1.1022  | val_0_accuracy: 0.75694 |  0:00:17s
epoch 6  | loss: 0.96771 | val_0_accuracy: 0.73611 |  0:00:20s
epoch 7  | loss: 0.89622 | val_0_accuracy: 0.70833 |  0:00:23s
epoch 8  | loss: 0.82075 | val_0_accuracy: 0.79514 |  0:00:26s
epoch 9  | loss: 0.7965  | val_0_accuracy: 0.80556 |  0:00:29s
epoch 10 | loss: 0.72889 | val_0_accuracy: 0.84028 |  0:00:32s
epoch 11 | loss: 0.75614 | val_0_accuracy: 0.84375 |  0:00:35s
epoch 12 | loss: 0.59688 | val_0_accuracy: 0.87847 |  0:00:38s
epoch 13 | loss: 0.57632 | val_0_accuracy: 0.89583 |  0:00:41s
epoch 14 | loss: 0.60825 | val_0_accuracy: 0.90972 |  0:00:44s
epoch 15 | loss: 0.6337  | val_0_accuracy: 0.92708 |  0

c:\venv_python_3.10\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [18]:
y_pred = tabnet_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[39  0  0  0  0  0  0  0  2  0]
 [ 0 40  1  0  0  0  1  0  0  0]
 [ 0  0 37  0  0  0  0  0  1  0]
 [ 0  0  0 31  0  1  0  0  0  0]
 [ 1  0  0  0 35  0  0  0  0  0]
 [ 0  0  0  0  0 27  0  0  0  1]
 [ 0  0  0  0  0  0 24  0  2  0]
 [ 0  0  0  0  0  0  0 42  0  0]
 [ 1  1  0  0  0  1  0  0 32  1]
 [ 0  1  0  0  0  0  0  0  1 37]]
Accuracy  0.9555555555555556


# XGBoost

In [19]:
num_round = 3
xgb_classifier = xgb.XGBClassifier()

xgb_parameters ={'max_depth' : [3,4,5,6] , 'n_estimators': [12,24,32], 
 'learning_rate' : [0.01,0.02,0.03],
 'gamma': [0.5, 1, 2], 'random_state':[99]}

In [20]:
# xgb_classifier=xgb_classifier.fit(x_train, y_train)

xgb_grid_search = GridSearchCV(xgb_classifier, xgb_parameters, cv=5, scoring="neg_mean_squared_error", verbose=2)
best_xgb_classifier = xgb_grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=24, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=24, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=24, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=24, random_state=99; to

In [21]:

xgb_grid_search.best_params_

{'gamma': 2,
 'learning_rate': 0.03,
 'max_depth': 5,
 'n_estimators': 24,
 'random_state': 99}

In [22]:
# y_pred = xgb_classifier.predict(x_test)
y_pred = best_xgb_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[38  1  0  0  0  0  0  0  2  0]
 [ 0 39  0  1  0  0  0  0  1  1]
 [ 0  0 37  0  0  0  0  0  0  1]
 [ 0  1  1 28  0  2  0  0  0  0]
 [ 0  0  0  0 34  1  1  0  0  0]
 [ 0  0  0  0  0 27  0  0  0  1]
 [ 0  0  0  1  2  0 23  0  0  0]
 [ 0  0  0  1  1  0  0 39  1  0]
 [ 0  2  0  1  0  0  0  1 31  1]
 [ 0  1  1  2  0  0  0  4  2 29]]
Accuracy  0.9027777777777778


# LightGBM

In [23]:
num_round = 3
lgbm_classifier = lgbm.LGBMClassifier()

In [24]:
lgbm_classifier=lgbm_classifier.fit(x_train, y_train)

In [25]:

y_pred = lgbm_classifier.predict(x_test)



print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[40  0  0  0  1  0  0  0  0  0]
 [ 0 40  0  1  0  0  0  0  0  1]
 [ 0  0 38  0  0  0  0  0  0  0]
 [ 0  0  0 29  0  2  0  0  1  0]
 [ 0  0  0  0 36  0  0  0  0  0]
 [ 0  0  0  0  1 27  0  0  0  0]
 [ 0  0  0  0  0  1 24  0  1  0]
 [ 0  0  0  0  0  0  0 42  0  0]
 [ 0  1  0  0  0  0  0  0 35  0]
 [ 0  1  0  0  0  0  0  0  1 37]]
Accuracy  0.9666666666666667


# CatBoost

In [26]:
num_round = 3
cbt_classifier = cbt.CatBoostClassifier()

In [27]:
cbt_parameters = {'depth'         : [3,4,5,6],
                'learning_rate' : [0.01,0.02,0.03],
                'n_estimators': [12,24,32], 
}

In [28]:
# cbt_classifier=cbt_classifier.fit(x_train, y_train)

cbt_grid_search = GridSearchCV(cbt_classifier, cbt_parameters, cv=5, scoring="neg_mean_squared_error", verbose=2)
best_cbt_classifier = cbt_grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 2.2884889	total: 147ms	remaining: 1.62s
1:	learn: 2.2723131	total: 149ms	remaining: 747ms
2:	learn: 2.2610038	total: 151ms	remaining: 453ms
3:	learn: 2.2444294	total: 152ms	remaining: 304ms
4:	learn: 2.2332898	total: 154ms	remaining: 215ms
5:	learn: 2.2230114	total: 155ms	remaining: 155ms
6:	learn: 2.2121249	total: 156ms	remaining: 111ms
7:	learn: 2.1957253	total: 157ms	remaining: 78.5ms
8:	learn: 2.1792675	total: 158ms	remaining: 52.7ms
9:	learn: 2.1664515	total: 159ms	remaining: 31.9ms
10:	learn: 2.1509423	total: 161ms	remaining: 14.6ms
11:	learn: 2.1403718	total: 162ms	remaining: 0us
[CV] END .......depth=3, learning_rate=0.01, n_estimators=12; total time=   0.1s
0:	learn: 2.2899113	total: 3.31ms	remaining: 36.4ms
1:	learn: 2.2785744	total: 4.72ms	remaining: 23.6ms
2:	learn: 2.2648771	total: 6.42ms	remaining: 19.3ms
3:	learn: 2.2505080	total: 7.85ms	remaining: 15.7ms
4:	learn: 2.2370859	total: 9.19ms	remaining: 

In [29]:
cbt_grid_search.best_params_

{'depth': 6, 'learning_rate': 0.03, 'n_estimators': 32}

In [30]:
# y_pred = cbt_classifier.predict(x_test)
y_pred = best_cbt_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[40  0  0  0  0  0  0  0  1  0]
 [ 0 35  1  1  0  0  0  0  5  0]
 [ 0  0 37  0  0  0  0  1  0  0]
 [ 0  1  1 27  0  2  0  0  1  0]
 [ 0  0  0  0 35  1  0  0  0  0]
 [ 0  0  0  0  0 27  0  0  0  1]
 [ 2  0  0  0  0  0 24  0  0  0]
 [ 0  0  0  0  0  0  0 42  0  0]
 [ 0  0  0  0  0  0  0  2 31  3]
 [ 0  2  0  0  0  0  0  2  1 34]]
Accuracy  0.9222222222222223


# Random Forest

In [31]:
rf_param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7],
    'criterion' :['gini', 'entropy']
}

In [32]:
num_round = 3
rf_classifier = RandomForestClassifier()

rf_grid_search = GridSearchCV(rf_classifier, rf_param_grid, cv=5, scoring="neg_mean_squared_error", verbose=2)
best_rf_classifier = rf_grid_search.fit(x_train, y_train)



Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END criterion=gini, max_depth=4, max_features=auto, n_estimators=200; total time=   0.3s


c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END criterion=gini, max_depth=4, max_features=auto, n_estimators=200; total time=   0.2s


...

c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END criterion=entropy, max_depth=7, max_features=auto, n_estimators=500; total time=   1.0s


c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END criterion=entropy, max_depth=7, max_features=auto, n_estimators=500; total time=   1.0s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=200; total time=   0.4s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=200; total time=   0.3s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=200; total time=   0.3s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=200; total time=   0.4s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=200; total time=   0.3s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=500; total time=   1.0s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=500; total time=   1.0s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=500; total time=   1.2s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=500; total time=   1.0s
[CV] END criterion=entropy, ma

c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [33]:
# rf_classifier = rf_classifier.fit(x_train, y_train)

In [34]:
best_rf_classifier.best_params_

{'criterion': 'entropy',
 'max_depth': 7,
 'max_features': 'auto',
 'n_estimators': 200}

In [35]:

y_pred = best_rf_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[40  0  0  0  1  0  0  0  0  0]
 [ 0 42  0  0  0  0  0  0  0  0]
 [ 0  0 38  0  0  0  0  0  0  0]
 [ 0  1  0 30  0  1  0  0  0  0]
 [ 0  0  0  0 36  0  0  0  0  0]
 [ 0  0  0  0  0 28  0  0  0  0]
 [ 1  0  0  0  0  1 24  0  0  0]
 [ 0  0  0  0  0  0  0 42  0  0]
 [ 0  0  0  0  0  0  0  0 36  0]
 [ 0  1  0  0  0  0  0  1  1 36]]
Accuracy  0.9777777777777777


# RVM(Relevance Vector Machine)

In [43]:
em_rvc = EMRVC(kernel="linear")
em_rvc = em_rvc.fit(x_train, y_train)


y_pred = em_rvc.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

# print(clf.predict(x_test))
# print(clf.predict_proba(X))
# print(clf.score(X, y))

Confusion Matrix
 [[40  0  0  0  1  0  0  0  0  0]
 [ 0 37  0  1  0  0  1  0  2  1]
 [ 0  0 38  0  0  0  0  0  0  0]
 [ 0  0  0 30  0  2  0  0  0  0]
 [ 0  0  0  0 36  0  0  0  0  0]
 [ 0  1  0  0  0 26  0  0  0  1]
 [ 0  0  0  0  2  0 24  0  0  0]
 [ 0  0  0  0  0  0  0 42  0  0]
 [ 0  1  0  0  0  0  1  0 34  0]
 [ 0  1  0  1  0  0  0  1  2 34]]
Accuracy  0.9472222222222222


In [44]:
em_rvc = EMRVC(kernel="rbf")
em_rvc = em_rvc.fit(x_train, y_train)


y_pred = em_rvc.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

c:\venv_python_3.10\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will change "
c:\venv_python_3.10\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will change "
c:\venv_python_3.10\lib\site-packages\sklearn_rvm\em_rvm.py:675: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  warnings.warn("The default value of gamma will change "
c:\venv_python_3.10\lib\site-packa

Confusion Matrix
 [[40  0  0  0  1  0  0  0  0  0]
 [ 0 39  0  1  0  0  0  0  2  0]
 [ 0  0 38  0  0  0  0  0  0  0]
 [ 0  0  0 29  0  2  0  0  1  0]
 [ 0  0  0  0 36  0  0  0  0  0]
 [ 0  0  0  0  0 27  0  0  0  1]
 [ 0  0  0  0  1  1 24  0  0  0]
 [ 0  0  0  0  0  0  0 42  0  0]
 [ 0  2  0  0  0  0  1  0 33  0]
 [ 0  1  0  1  1  0  0  0  0 36]]
Accuracy  0.9555555555555556
